In [1]:
import pandas as pd, numpy as np, os, sys
import matplotlib.pyplot as plt
import seaborn as sns

from funcs import update_vienne
from mpf_funcs import update_caledonie, lowercase_column, convert_date_format

import similarity_cython


In [2]:
df = pd.read_csv('data/primary/big_bertha.csv', sep=',')
df_other = pd.read_csv('data/primary/edited_geotagged_bertha.csv', sep=',') #On recupure l'information geocodée
df = pd.merge(df, df_other[['pays']], left_index=True, right_index=True)
df = df.apply(update_vienne, axis=1)
df = df.apply(update_caledonie, axis=1)
lowercase_column(df, 'Nom du soldat')
lowercase_column(df, 'Prénom(s)')



In [3]:
# CSV des mpfs  de la Seine
mpf = pd.read_csv('data/primary/mpf_seine.csv', sep=';')
lowercase_column(mpf, 'nom')
lowercase_column(mpf, 'prenom')
mpf['naissance_jour_mois_annee'] = mpf['naissance_jour_mois_annee'].apply(lambda x : convert_date_format(x) if isinstance(x, str) else x)


/var/folders/41/1r7_f1_93g56d738vfqw0y8m0000gn/T/ipykernel_58717/3078710691.py:2: DtypeWarning: Columns (2,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  mpf = pd.read_csv('data/primary/mpf_seine.csv', sep=';')


In [4]:
merged_df = pd.merge(df, mpf,
                     left_on=['Date de naissance'],
                     right_on=['naissance_jour_mois_annee'],
                     how='inner')
merged_df.dropna(axis=1, how='all', inplace=True)

In [5]:
# Calculate similarity scores for first and last names
merged_df['score_firstname'] = merged_df.apply(lambda row: similarity_cython.similarity_score_cython(str(row['Prénom(s)']).encode('utf-8'), str(row['prenom']).encode('utf-8')), axis=1)
merged_df['score_lastname'] = merged_df.apply(lambda row: similarity_cython.similarity_score_cython(str(row['Nom du soldat']).encode('utf-8'), str(row['nom_y']).encode('utf-8')), axis=1)

matches = merged_df[(merged_df['score_firstname'] > 75) & (merged_df['score_lastname'] > 65)]

In [6]:
matches.to_csv('~/dev/projet_memoire2/data/primary/merged_mpf.csv', sep=',', index=False)